In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory
import os
import sys

module_path = os.path.abspath(os.path.join(".."))

if module_path not in sys.path:
    sys.path.append(module_path)

sys.executable

'/home/danian/udemy/vamcess/bin/python'

In [2]:
from amcess.molecule import Molecule
from amcess.cluster import Cluster
from amcess.search_engine import SearchConfig

In [3]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 
# 3Dmol.js viewer in a notebook.
import py3Dmol

def visualize(
        xyz_file: str, 
        sphere_radius: int, 
        sphere_center: tuple,
        style='sphere',
        ):
        """
        Visualize the molecule with a sphere around the molecule.
        """

        sr = sphere_radius
        cx, cy, cz = sphere_center

        if not xyz_file:
                return None

        # starting visualization
        xyz_view = py3Dmol.view() # width=400, height=300)#, linked=False, viewergrid=(2,2))
        xyz_view.addModelsAsFrames(xyz_file,'xyz')
        # xyz_view.setStyle({style: {'radius': 0.6}})

        if style == 'stick':
                xyz_view.setStyle({'stick': {'radius': 0.15}})
        else:
                xyz_view.setStyle({'sphere': {'radius': 0.8}})


        xyz_view.zoomTo()

        # animation base on several XYZ coordinates snapshots
        # xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})
        xyz_view.animate({'loop': 'backAndForth', 'speed': 1})


        # sphere center
        sc = {"x": cx, "y": cy, "z": cz}

        # 20% to check visually that everything is inside
        xyz_view.addSphere({'center': sc, 
                'radius': sr, 
                'color' :'yellow',
                'alpha': 0.5,
                })

        # cartesian 3D axes
        x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
        y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
        z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

        xyz_view.addLine(x_axis)
        xyz_view.addLine(y_axis)
        xyz_view.addLine(z_axis)

        xyz_view.addLabel("x", {
                'position':x_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("y", {
                'position':y_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("z", {
                'position':z_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })

        return xyz_view

In [4]:
h = {
    "atoms": [
        ("H", 0., 0., 0.74),
        ("H", 0., 0., 0.),    ],
    "charge": 0,
    "multiplicity": 2,
}

hydrogen = Cluster(h, h)
hydrogen.sphere_radius = 1.5
hydrogen.sphere_center = (0, 0, 0)

# avoiding atomic overlapping
hydrogen = hydrogen.initialize_cluster(max_closeness=0.5)


In [5]:
xyzfile_ascec = "configurations_ascec.xyz"
# Define a search configuration method to optimize the Energy of Cluster HF using ASCEC (default search_methodology='ASCEC')
SC_ascec = SearchConfig(
    hydrogen, basis="sto-3g", methodology="HF", outxyz=xyzfile_ascec
)
# Define number of initial evaluations and maximum number of iterations
SC_ascec.run(maxCycles=10, number_temperatures=2)
print(f"Minima found: {SC_ascec._search} Hartree")

*** Minimization: ASCEC ***


 Current temperature  810.00 K, progress: 100.00%, with    2 configurations accepted (cycle    0/10  )Minima found: <amcess.ascec.Ascec object at 0x7fabf68d2950> Hartree


In [6]:
xyz_file = open("configurations_ascec.xyz", "r").read()

view = visualize(
    xyz_file=xyz_file,
    sphere_radius=hydrogen.sphere_radius * 1.5,
    sphere_center=hydrogen.sphere_center,
    style="stick",
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
water = {
    "atoms": [
        ("O", 0, 0, 0),
        ("H", 0.58708, 0.75754, 0),
        ("H", -0.58708, 0.75754, 0),
    ],
    "charge": 0,
    "multiplicity": 1,
}

water_dimer = Cluster(water, water)
water_dimer.sphere_radius = 1.5
water_dimer.sphere_center = (0, 0, 0)

w4 = Cluster(2 * water_dimer)
w4.sphere_radius = 2.0
w4.sphere_center = (0, 0, 0)

# avoiding atomic overlapping
w4 = w4.initialize_cluster(max_closeness=1.0)

In [11]:
xyzfile_ascec = 'configurations_ascec.xyz'
# Define a search configuration method to optimize the Energy of Cluster HF using ASCEC (default search_methodology='ASCEC')
SC_ascec = SearchConfig(water_dimer,  basis='sto-3g', outxyz=xyzfile_ascec)
# Define number of initial evaluations and maximum number of iterations
SC_ascec.run(maxCycles=10, number_temperatures=2)
#print(f'Minima found: {SC_ascec._search.run} Hartree')

*** Minimization: ASCEC ***
*** Exception in SCF Calculation 

*** Exception in SCF Calculation 

 Current temperature  810.00 K, progress: 100.00%, with    2 configurations accepted (cycle    0/10  )

In [13]:
xyz_file = open("configurations_ascec.xyz", "r").read()

view = visualize(
    xyz_file=xyz_file,
    sphere_radius=w4.sphere_radius * 1.2,
    sphere_center=w4.sphere_center,
    # style="stick",
    style="sphere",
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol